In [2]:
import cfscrape
from lxml import etree
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import re
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from bs4 import BeautifulSoup
import re
from random import randint
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pickle
import requests
import math
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pyperclip 

In [161]:
with open('Other-Grantees-LinkedIn.pkl', 'rb') as f:
    others_batch = pickle.load(f)
others_batch.shape

(12061, 6)

In [162]:
batch = (9000,10000)

In [163]:
others_batch = others_batch.loc[batch[0]:batch[1]]
others_batch.shape

(1001, 6)

In [164]:
others_batch.head(1)

,Grantees,Index,Found Names,Found Links,Found Results,Searching Name
9000,"WAUGH, JANET H",10839,NaN,NaN,NaN,NaN


# Names Pool Light Search

In [129]:
def get_light_linkedin(start, end, data, file, instr_path):
    for i in range(start, end): 
        
        name = data['Grantees'].loc[i]
        new_name = re.sub(r'( TRUSTEE| LLC| EST| OF| TRUST| INC| LTDA)','', name)
        new_name = re.sub(r'(( \w)+$)','', new_name)
        new_name = re.sub(r'( \w )',' ', new_name)
        new_name = re.sub(r'(\, )',' ', new_name)

        data['Searching Name'].loc[i] = new_name

        if 'marriott' in new_name.lower():
            continue

        browser.find_element_by_xpath(instr_path).send_keys(new_name)

        browser.implicitly_wait(0.6)
        browser.find_element_by_xpath(instr_path).send_keys(Keys.ENTER)
        print(i, new_name)

        sleep(randint(1,2)) 
        actions = ActionChains(browser)
        for _ in range(3):
            actions.send_keys(Keys.PAGE_DOWN).perform()
            sleep(0.1)

        
        sleep(0.6)
        html = browser.page_source
        soup = BeautifulSoup(html, 'html.parser')

        sleep(0.3)
        found_names = []
        els = soup.findAll('span', attrs={'class':'name actor-name'})
        for a in els:
            found_names.append(a.text)

        if len(found_names) == 0:
            found_names='Not found'
            print('Not found')
        elif len(found_names) > 5:
            print(', '.join(found_names[:5]))
        else:
            print(', '.join(found_names))

        data['Found Names'].loc[i] = found_names

        sleep(0.3)
        found_links = []   
        els = soup.findAll('a', attrs={'class':'search-result__result-link ember-view'})
        for a in els:
            if a['href'] not in found_links:
                found_links.append(a['href'])
        if len(found_links) > 0:
            data['Found Links'].loc[i] = found_links

        # results
        try:
            results = browser.find_element_by_xpath(path_results).text
            print(results, '\n')
            data['Found Results'].loc[i] = results
        except:
            data['Found Results'].loc[i] = np.nan

        browser.find_element_by_xpath(instr_path).clear()

        if i%50==0 or i==end-1:
            sleep(randint(5,15))
            print(i,'Saved')
        with open(file, 'wb') as f:
            pickle.dump(data, f)
        
        # check limit
        try:
            limit_path = '/html/body/div[9]/div[4]/div/div/h1'
            limit = browser.find_element_by_xpath(limit_path).text
            if limit:
                print('Limit is reached')
                break
        except:
            pass

In [165]:
file = 'Other-Grantees-LinkedIn-Batch_{}-{}.pkl'.format(batch[0], batch[1])
path_results = '/html/body/div[9]/div[4]/div/div[2]/div/div[2]/div/div/div/div/h3'
instr_path = '/html/body/header/div/form/div/div/div/div/div[1]/div/input' 

In [166]:
file, batch

('Other-Grantees-LinkedIn-Batch_9000-10000.pkl', (9000, 10000))

In [169]:
%%time
get_light_linkedin(batch[0]+60, batch[1], others_batch, file, instr_path)

9060 WILDE SARA
Not found
9061 ALMEIDA THOMAS
Thomas Almeida, Anita Almeida, Thomas Almeida, CFA, Jonathan C. Almeida, Thomas Almeida
135 results 

9062 JOHN CALLAHAN
john callahan, John Callahan, John Callahan, John Callahan, John Callahan
About 1,600 results 

9063 JULIA CALLAHAN
Julia Callahan, Julie Callahan, Julia Callahan, Julie Callahan, Julie Callahan
198 results 

9064 CALLAHAN JOHN
john callahan, John Callahan, John Callahan, John Callahan, John Callahan
About 1,600 results 

9065 CALLAHAN JULIA
Julia Callahan, Julie Callahan, Julia Callahan, Julie Callahan, Julie Callahan
198 results 

9066 KEY JOE
Joseph Key, Joseph Key, PHR, Joe Lonsdale, Joseph Key, Joseph Key
About 60,000 results 

9067 NEWCOM CYNTHIA
cindy newcom, Cindy Newcom
2 results 

9068 NEWCOM MARK
Mark Newcom, Mark Turner
11 results 

9069 JOHNSON RAYMOND JR
Raymond Johnson Jr, P. Raymond Johnson JR., Raymond Johnson, Jr, Raymond J. Almeida Jr., Raymond Johnson Jr
210 results 

9070 JOHNSON HOLLIS
Hollis Johnson

Donna Chandler, Donna Chandler, Donna Perkins, Donna Chandler, CFRE, Donna Chandler
632 results 

9168 UPHOLZER STEVEN
Not found
9169 UPHOLZER ANGEL
Not found
9170 JOHNSTON TERRY JR
Terry W. Johnston Jr., Terry Fife, Jr, Terry Johnson Johnson Jr, Michael H. Johnston, MPS, Terence Daniels Jr.
24 results 

9171 JOHNSTON JEANNETTE
Jeannette Johnston, Jeannette Johnston, Jeannette Johnston, jeannette johnston, Jeannette Johnston
690 results 

9172 KINDER AMANDA
Amanda Hill, Amanda Taylor, Amanda Kinder, Amanda Guthrie, Amanda Galvan
180 results 

9173 KINDER BRIAN
Brian Story, Brian Kinder, Brian Kinder, Brian Kinder, Brian Brister
271 results 

9174 SHERMOEN STEVEN
Steve Shermoen, Steve Shermoen, Steven Shermoen, Steve Shermoen, Steve Shermoen
6 results 

9175 SNYDER THOMAS
Donald Thomas Snyder, Thomas Snyder, Thomas Snyder, Thomas Snyder, Thomas Snyder
About 2,100 results 

9176 SNYDER CHRISTINA
Christina Snyder, Christina Snyder, Christina Snyder, Christina Snyder, Christina Snyder, CFA

554 results 

9272 CASPER MARY ANN
Mary Ann Casper, Ann Casper, Mary Ann Casper, Mary Ann Casper, George Ann Casper
Showing 62 results 

9273 MITCHELL KENNETH
Kenneth Mitchell, Kenneth Mitchell, Kenneth Mitchell, Kenneth Mitchell, Kenneth Mitchell
About 1,700 results 

9274 SAUL ELAINE
Elaine Saul, Elaine Saul, Elaine Sarkis, Elaine Saul
33 results 

9275 MCMAHAN JENNIFER
Jennifer McMahan, Jennifer McMahan, Jennifer McMahan, Jennifer McMahan, Jennifer and Chonzie McMahan
287 results 

9276 WEST CHRISTOPHER
Christopher West, Christopher West, Christopher West, Chris West, Christopher West
About 52,000 results 

9277 WEST REBECCA
Rebecca West, Rebecca West, Rebecca West, Rebecca West, Rebecca West
About 16,000 results 

9278 DIBBLE STEPHEN
Stephen Dibble, Steven Dibble, Steve Dibble, Stephen Dibble, Steve Dibble
77 results 

9279 HESS DAVID
David Hess, david hess, David Hess, David Hess, David Hess
About 1,500 results 

9280 HESS MARTHA
Martha Hess, Martha Hess, Martha Hess, Martha H., m

John Hendricks, John Weigel, John Weigel
241 results 

9378 WEIGEL SHEILA
Sheila Weigel, Sheila Weigel
3 results 

9379 JAMES PORTER LIVING
james porter, James Porter, jamie porter
307 results 

9380 PORTER JAMES
James Porter, James Porter, James Porter
About 5,600 results 

9381 JONES MICHAEL
Michael S. Jones, Michael Jones, Michael Jones, Michael Jones, Michael Jones
About 30,000 results 

9382 LAFATA JONES JEAN
Jean Lafata Jones, Jean LaFata
2 results 

9383 ABATEMARCO SANDRA
Not found
9384 ADY NANCY
Nancy Ady, Ady Sanchez-Macarty, Nancy Ady
3 results 

9385 NANCY ADY
Nancy Ady, Ady Sanchez-Macarty, Nancy Addy
8 results 

9386 BEYER STEVEN
Steve Beyer, Steve Beyer, Steve Beyer
284 results 

9387 BEYER ELENI
Not found
1 result 

9388 MILLIGAN DONALD
Donald Milligan, Don Milligan, Donald M., Don Milligan, DON MILLIGAN
135 results 

9389 MILLIGAN GLENDA
Glenda Milligan, Glenda Milligan, Glenda Milligan, Glenda Milligan, Glenda Gilbertson Milligan
7 results 

9390 WARD CHELSEA
Chelsea K

9502 MICHALSKI JOHN
John Michalski, John Michalski, John Michalski
129 results 

9503 MICHALSKI PATRICIA
Patty Michalski, Patricia Michalski, Patricia Michalski
24 results 

9504 AUSTIN BETTY
Not found


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/header/div/form/div/div/div/div/div[1]/div/input"}
  (Session info: chrome=83.0.4103.61)


In [170]:
with open(file, 'wb') as f:
    pickle.dump(data, f)